In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print("{0}\t{1}".format(word.lower(), 1))

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print("{0}\t{1}".format(current_key, word_sum))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print("{0}\t{1}".format(current_key, word_sum))

Appending to reducer.py


In [4]:
%%writefile reverse_mapper.py

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        print("{0}\t{1}".format(count, word))
    except ValueError as e:
        continue

Overwriting reverse_mapper.py


In [5]:
%%writefile reverse_reducer.py

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
        print("{0}\t{1}".format(word, count))
    except ValueError as e:
        continue

Overwriting reverse_reducer.py


In [14]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [7]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR_FINAL="wordcount_final_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_FINAL} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options=-nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files reverse_mapper.py,reverse_reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_FINAL} > /dev/null

hdfs dfs -ls ${OUT_DIR_FINAL}


rm: `wordcount_result_1512431128069491': No such file or directory
17/12/04 23:45:31 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/04 23:45:31 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/04 23:45:32 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/04 23:45:32 INFO mapreduce.JobSubmitter: number of splits:2
17/12/04 23:45:32 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1512399025937_0031
17/12/04 23:45:32 INFO impl.YarnClientImpl: Submitted application application_1512399025937_0031
17/12/04 23:45:32 INFO mapreduce.Job: The url to track the job: http://974762bfb744:8088/proxy/application_1512399025937_0031/
17/12/04 23:45:32 INFO mapreduce.Job: Running job: job_1512399025937_0031
17/12/04 23:45:37 INFO mapreduce.Job: Job job_1512399025937_0031 running in uber mode : false
17/12/04 23:45:37 INFO mapreduce.Job:  map 0% reduce 0%
17/12/04 23:45:53 INFO mapreduce.Job:  map 58% reduce 0%
17/12/04 23:4